In [26]:
from datasets import Dataset
import requests
import time
import numpy as np
import tensorflow as tf
import pandas as pd
import os


# API endpoint
url = "https://db.ygoprodeck.com/api/v7/cardinfo.php"

# Request parameters
params = {

}

word_list = dict()
subtype_list = dict()
char_list = dict()
card_number_listed = 0

try:
    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        cards = pd.DataFrame(data=data["data"])
        
except requests.exceptions.RequestException as e:
    print("An error occurred:", e)

In [27]:
# Save the dataframe as csv
cards.to_csv("deck.csv", index=False)

In [28]:
import datasets
from PIL import Image



def read_image(file_path):
    with Image.open(file_path) as img:
        return img.convert('RGB')

df_c = cards[["id", "name"]].head(10)
df_c['image'] = df_c.apply(lambda row: f"images/{row['id']}.jpg", axis=1)
df_c['image_data'] = df_c['image'].apply(lambda x: read_image(os.path.join('', x)))
df_c

,id,name,image,image_data
0,34541863,"""A"" Cell Breeding Device",images/34541863.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
1,64163367,"""A"" Cell Incubator",images/64163367.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
2,91231901,"""A"" Cell Recombination Device",images/91231901.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
3,73262676,"""A"" Cell Scatter Burst",images/73262676.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
4,98319530,"""Infernoble Arms - Almace""",images/98319530.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
5,37478723,"""Infernoble Arms - Durendal""",images/37478723.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
6,64867422,"""Infernoble Arms - Hauteclere""",images/64867422.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
7,90861137,"""Infernoble Arms - Joyeuse""",images/90861137.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
8,44256816,1st Movement Solo,images/44256816.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...
9,86988864,3-Hump Lacooda,images/86988864.jpg,<PIL.Image.Image image mode=RGB size=624x624 a...


In [ ]:
def load_image(row):
    feature = datasets.Image(decode=False)
    try:
        image = Image.open(row['image']).convert("RGB")
    except:
        print(f"Failed to open image {row['id']}")
        return


    return {'image': feature.encode_example(image)}

In [29]:
import base64
from io import BytesIO

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

df_c["image_data"] = df_c["image_data"].apply(image_to_base64)
huggingface_dataset = Dataset.from_pandas(df_c)

In [30]:
print(df_c['image_data'][0]['image'])

TypeError: string indices must be integers

In [25]:
from datasets import Dataset

huggingface_dataset = Dataset.from_pandas(df_c)
huggingface_dataset = huggingface_dataset.cast(column_types={"id": int, "name": str})


ArrowInvalid: ('Could not convert <PIL.Image.Image image mode=RGB size=624x624 at 0x22CF120ADA0> with type Image: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column image_data with type object')

In [33]:
df_c

,id,name,image,image_data
0,34541863,"""A"" Cell Breeding Device",images/34541863.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,64163367,"""A"" Cell Incubator",images/64163367.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,91231901,"""A"" Cell Recombination Device",images/91231901.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,73262676,"""A"" Cell Scatter Burst",images/73262676.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,98319530,"""Infernoble Arms - Almace""",images/98319530.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
5,37478723,"""Infernoble Arms - Durendal""",images/37478723.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
6,64867422,"""Infernoble Arms - Hauteclere""",images/64867422.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
7,90861137,"""Infernoble Arms - Joyeuse""",images/90861137.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
8,44256816,1st Movement Solo,images/44256816.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
9,86988864,3-Hump Lacooda,images/86988864.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [24]:
from datasets import Dataset, load_dataset
from huggingface_hub import HfApi

dataset = Dataset.from_pandas(df_c)
dataset

# Upload to Huggingface
dataset.push_to_hub("FabioArdi/test", private=True)

ArrowInvalid: ('Could not convert <PIL.Image.Image image mode=RGB size=624x624 at 0x22CF120ADA0> with type Image: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column image_data with type object')

In [32]:
# Upload to Huggingface
dataset.push_to_hub("FabioArdi/test", private=True)

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:03<00:00,  3.17s/it]
